In [1]:
%reload_kedro

[08/04/24 23:44:46] INFO     Resolved project path as: /Users/sharkovgrigory/code/fraud_detection.  ]8;id=125944;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=396980;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/kedro/ipython/__init__.py#171\171]8;;\
                             To set a different path, run '%reload_kedro <project_root>'                           

                    INFO     Registering new custom resolver: 'km.random_name'                    ]8;id=204574;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/kedro_mlflow/framework/hooks/mlflow_hook.py\mlflow_hook.py]8;;\:]8;id=329337;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/kedro_mlflow/framework/hooks/mlflow_hook.py#65\65]8;;\

                    WARNING  No 'mlflow.yml' config file found in environment. Default            ]8;id=696228;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/kedro_mlflow/framework/hooks/mlflow_hook.py\mlflow_hook.py]8;;\:]8;id=604184;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/kedro_mlflow/framework/hooks/mlflow_hook.py#78\78]8;;\
                             configuration will be used. Use ``kedro mlflow init`` command in CLI                  
                             to customize the configuration.                                                       

                    INFO     The 'tracking_uri' key in mlflow.yml is relative            ]8;id=826187;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/kedro_mlflow/config/kedro_mlflow_config.py\kedro_mlflow_config.py]8;;\:]8;id=984789;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/kedro_mlflow/config/kedro_mlflow_config.py#260\260]8;;\
                             ('server.mlflow_(tracking|registry)_uri = mlruns'). It is                             
                             converted to a valid uri:                                                             
                             'file:///Users/sharkovgrigory/code/fraud_detection/mlruns'                            

                    INFO     Kedro is sending anonymous usage data with the sole purpose of improving ]8;id=384154;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/kedro_telemetry/plugin.py\plugin.py]8;;\:]8;id=369756;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/kedro_telemetry/plugin.py#233\233]8;;\
                             the product. No personal data or IP addresses are stored on our side. If              
                             you want to opt out, set the `KEDRO_DISABLE_TELEMETRY` or `DO_NOT_TRACK`              
                             environment variables, or create a `.telemetry` file in the current                   
                             working directory with the contents `consent: false`. Read more at                    
                             https://docs.kedro.org/en/stable/configuration/telemetry.html                         

[08/04/24 23:44:47] INFO     Kedro project fraud_detection                                          ]8;id=176056;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=796813;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/kedro/ipython/__init__.py#141\141]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=297581;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=909085;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/kedro/ipython/__init__.py#142\142]8;;\
                             'pipelines'                                                                           

                    INFO     Registered line magic 'run_viz'                                        ]8;id=603483;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=64376;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/kedro/ipython/__init__.py#148\148]8;;\

In [2]:
from fraud_detection.utils.encoder_utils import get_row_preprocessor, FAST_MODEL_SETTINGS

In [3]:
train_data = catalog.load("combined_train_data")
id_cols = context.params.get("train_set_parameters").get("columns").get('id_columns')
cat_cols = ["merchant", "category", "gender"]
train_data.head()

[08/04/24 23:44:49] INFO     Loading data from combined_train_data (CSVDataset)...              ]8;id=695887;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=103098;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/kedro/io/data_catalog.py#508\508]8;;\

,cc_num,window_id,label,merchant,category,amt,gender,city,state,city_pop,job
0,60416207185,60416207185_115,True,"fraud_Schroeder, Hauck and Treutel",entertainment,90.60,F,Fort Washakie,WY,1645,Information systems manager
1,60416207185,60416207185_115,True,"fraud_Hills, Hegmann and Schaefer",health_fitness,26.13,F,Fort Washakie,WY,1645,Information systems manager
2,60416207185,60416207185_115,True,"fraud_Langworth, Boehm and Gulgowski",shopping_net,1.24,F,Fort Washakie,WY,1645,Information systems manager
3,60416207185,60416207185_115,True,fraud_Turcotte-Halvorson,misc_pos,11.60,F,Fort Washakie,WY,1645,Information systems manager
4,60416207185,60416207185_115,True,fraud_Padberg-Sauer,home,200.83,F,Fort Washakie,WY,1645,Information systems manager


In [4]:
preprocessor = get_row_preprocessor(cat_cols, FAST_MODEL_SETTINGS)

In [5]:
preprocessor.fit(train_data)
# preprocessor.transform(train_data)

Pipeline(steps=[('combined_embeddings',
                 FeatureUnion(transformer_list=[('emb_merchant',
                                                 FastTextEncoder(column='merchant',
                                                                 embedder_settings={'min_count': 3,
                                                                                    'vector_size': 3,
                                                                                    'window': 5,
                                                                                    'workers': 4})),
                                                ('emb_category',
                                                 FastTextEncoder(column='category',
                                                                 embedder_settings={'min_count': 3,
                                                                                    'vector_size': 3,
                                                                                    'window': 5,
                                                                                    'workers': 4})),
                                                ('emb_gender',
                                                 FastTextEncoder(column='gender',
                                                                 embedder_settings={'min_count': 3,
                                                                                    'vector_size': 3,
                                                                                    'window': 5,
                                                                                    'workers': 4}))]))])

In [7]:
import pandas as pd
pd.DataFrame(
    preprocessor.transform(train_data.loc[:2]),
    columns=['emb_merchant', 'emb_category', 'emb_gender']
).to_dict()


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 import pandas as pd                                                                          │
│ ❱ 2 pd.DataFrame(                                                                                │
│   3 │   preprocessor.transform(train_data.loc[:2]),                                              │
│   4 │   columns=['emb_merchant', 'emb_category', 'emb_gender']                                   │
│   5 ).to_dict()                                                                                  │
│                                                                                                  │
│ /opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/pandas/core/frame.py:827 in    │
│ __init__                                                                                         │
│                                                                                                  │
│     824 │   │   │   │   │   copy=_copy,                                                          │
│     825 │   │   │   │   )                                                                        │
│     826 │   │   │   else:                                                                        │
│ ❱   827 │   │   │   │   mgr = ndarray_to_mgr(                                                    │
│     828 │   │   │   │   │   data,                                                                │
│     829 │   │   │   │   │   index,                                                               │
│     830 │   │   │   │   │   columns,                                                             │
│                                                                                                  │
│ /opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/pandas/core/internals/construc │
│ tion.py:336 in ndarray_to_mgr                                                                    │
│                                                                                                  │
│    333 │   │   values.shape[0], values.shape[1], index=index, columns=columns                    │
│    334 │   )                                                                                     │
│    335 │                                                                                         │
│ ❱  336 │   _check_values_indices_shape_match(values, index, columns)                             │
│    337 │                                                                                         │
│    338 │   if typ == "array":                                                                    │
│    339 │   │   if issubclass(values.dtype.type, str):                                            │
│                                                                                                  │
│ /opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/pandas/core/internals/construc │
│ tion.py:420 in _check_values_indices_shape_match                                                 │
│                                                                                                  │
│    417 │   │                                                                                     │
│    418 │   │   passed = values.shape                                                             │
│    419 │   │   implied = (len(index), len(columns))                                              │
│ ❱  420 │   │   raise ValueError(f"Shape of passed values is {passed}, indices imply {implied}")  │
│    421                                                                                           │
│    422                                                     

In [12]:
lst_a = [1,2,3]
lst_a[:2]

[1, 2]

In [15]:
    preprocessor.transform(train_data.loc[:5]),



(
    array([array([-0.538851  ,  0.22603846,  0.28873155, -3.3838582 ,  2.72914   ,
              -2.2270808 , -0.01915043,  0.29478744,  0.2175423 , -4.805849  ,
               3.179892  , -2.308592  ,  0.        ,  0.        ,  0.        ],
             dtype=float32)                                                    ,
       array([-5.020727  ,  1.6833534 ,  0.14954275, -2.2604127 ,  0.3970176 ,
               3.7264507 , -0.01915043,  0.29478744,  0.2175423 , -3.689961  ,
               0.88341385,  1.9629366 ,  0.        ,  0.        ,  0.        ],
             dtype=float32)                                                    ,
       array([-0.09547748,  0.3529062 , -0.00431618,  0.69908535,  0.5669764 ,
               2.817177  , -0.01915043,  0.29478744,  0.2175423 ,  2.6613328 ,
               2.1994588 , -0.57130235,  0.        ,  0.        ,  0.        ],
             dtype=float32)                                                    ,
       array([ 0.23025572,  3.683823

In [73]:
sentence_embeddings = []
for sentence in train_data.merchant[:2]:
    sentence_embeddings.append(embedder.transform_sentence(sentence))

sentence_embeddings


[
    array([ 0.03637635,  0.5257791 ,  0.39206585, -0.88935256,  0.94260186,
       -0.8020142 ,  0.16439702,  0.36078495,  0.21776949, -5.9001684 ,
        2.2033362 , -1.9886831 ,  0.        ,  0.        ,  0.        ],
      dtype=float32),
    array([-5.591079  ,  1.0973794 ,  0.5489118 , -1.3504852 ,  0.37101763,
        3.8094745 ,  0.16439702,  0.36078495,  0.21776949, -2.0842116 ,
        0.33817062,  2.190501  ,  0.        ,  0.        ,  0.        ],
      dtype=float32)
]

In [ ]:
columns = ["merchant", "category"]
text_embedders = []
for col in columns:
    _embedder = TextEmbedder(train_data[col], fast_model_settings)
    text_embedders.append(_embedder)